![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successf

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [7]:
def clone_repo(repo_url):

  repo_name = repo_url.split("/")[-1]
  repo_path = f"/content/{repo_name}"
  Repo.clone_from(repo_url, str(repo_path))
  return str(repo_name)

In [8]:
path = clone_repo("https://github.com/CoderAgent/SecureAgent")

In [ ]:
path = "/content/SecureAgent"

In [9]:
SUPPORTED_EXTENSIONS = [".py", ".js", ".tsx", ".ts", ".java", ".cpp"]

IGNORED_DIRS = [".git", "node_modules", "dist", "__pycache__", ".next", ".vscode", "env"]

In [10]:
def get_file_content(file_path, repo_path):
    try:
      with open(file_path, "r", encoding="utf-8") as f:
          content = f.read()
          rel_path = os.path.relpath(file_path, repo_path)

          return {
              "name": rel_path,
              "content": content
          }

    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def get_main_files_content(repo_path: str):
   """
   Get content of supported code files from the local repository.


   Args:
       repo_path: Path to the local repository


   Returns:
       List of dictionaries containing file names and contents
   """
   files_content = []


   try:
       for root, _, files in os.walk(repo_path):
           # Skip if current directory is in ignored directories
           if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
               continue


           # Process each file in current directory
           for file in files:
               file_path = os.path.join(root, file)
               if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                   file_content = get_file_content(file_path, repo_path)
                   if file_content:
                       files_content.append(file_content)


   except Exception as e:
       print(f"Error reading repository: {str(e)}")


   return files_content





In [11]:
file_content = get_main_files_content(path)

In [12]:
file_content

[{'name': 'src/constants.ts',
  'content': 'import { Node } from "@babel/traverse";\nimport { JavascriptParser } from "./context/language/javascript-parser";\nimport { ChatCompletionMessageParam } from "groq-sdk/resources/chat/completions";\n\nexport interface PRFile {\n  sha: string;\n  filename: string;\n  status:\n    | "added"\n    | "removed"\n    | "renamed"\n    | "changed"\n    | "modified"\n    | "copied"\n    | "unchanged";\n  additions: number;\n  deletions: number;\n  changes: number;\n  blob_url: string;\n  raw_url: string;\n  contents_url: string;\n  patch?: string;\n  previous_filename?: string;\n  patchTokenLength?: number;\n  old_contents?: string;\n  current_contents?: string;\n}\n\nexport interface BuilderResponse {\n  comment: string;\n  structuredComments: any[];\n}\n\nexport interface Builders {\n  convoBuilder: (diff: string) => ChatCompletionMessageParam[];\n  responseBuilder: (feedbacks: string[]) => Promise<BuilderResponse>;\n}\n\nexport interface PatchInfo {\

# Embeddings

In [13]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [14]:
text = "I am a programmer"
embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
embeddings

array([ 1.81737654e-02, -3.02657508e-03, -4.77465875e-02,  1.86379403e-02,
        3.14537995e-02,  1.87255293e-02, -1.52534274e-02, -6.77293688e-02,
       -1.26903653e-02,  1.28427576e-02,  5.80701306e-02,  4.00234833e-02,
        3.27073298e-02,  7.12998435e-02,  5.56373373e-02,  1.68628506e-02,
        6.97603747e-02, -5.02619930e-02,  6.13140827e-03, -1.46559235e-02,
       -4.51957993e-03,  4.82934639e-02, -2.53051296e-02, -1.97862904e-03,
       -4.36902530e-02, -2.41507161e-02,  1.29505759e-02, -3.78611824e-03,
       -2.05718316e-02,  1.09819308e-01,  3.07672890e-03, -2.80443169e-02,
       -1.55807249e-02, -1.24789868e-02,  1.75239131e-06, -2.93756695e-03,
       -1.43048428e-02,  4.88386713e-02, -6.21114224e-02,  2.95061413e-02,
       -1.40470508e-02,  2.20708270e-02,  1.13067888e-02,  4.70893271e-02,
        7.58305984e-03, -8.30314530e-05,  6.67821169e-02, -1.21320095e-02,
        4.39386303e-03,  2.47453637e-02,  1.02529004e-02, -6.54432410e-03,
       -5.53147821e-03, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [16]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [17]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-17-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-17-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [19]:
# Insert the codebase embeddings into Pinecone

documents= []

for file in file_content:
  doc = Document(
      page_content=f"{file['name']}\n{file['content']}",
      metadata={
          "source": file["name"]
      }
  )
  documents.append(doc)

  vectorstore = PineconeVectorStore.from_documents(
      documents=documents,
      embedding=HuggingFaceEmbeddings(),
      index_name="codebase-rag",
      namespace="https://github.com/CoderAgent/SecureAgent"
  )



<ipython-input-19-66f7a22e0de0>:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [20]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [21]:
query = "How is the Javascript parser used?"

In [22]:
query_embedding = get_huggingface_embeddings(query)

In [23]:
query_embedding

array([ 5.71991131e-02, -3.48081291e-02, -3.27215418e-02,  5.29010817e-02,
       -3.88054959e-02,  2.21033078e-02,  1.60938818e-02, -1.00735975e-02,
        3.04608904e-02, -6.25874922e-02,  2.71743089e-02,  3.67565900e-02,
        5.69119453e-02,  5.45354113e-02,  4.02487814e-02, -4.73744869e-02,
        3.59910820e-03,  6.65521948e-03,  1.47536704e-02,  3.57538760e-02,
        1.81228686e-02,  1.24748386e-02, -2.07926128e-02,  6.99328259e-02,
       -1.78905539e-02, -1.98267922e-02, -8.77424423e-03, -4.04769043e-03,
       -4.82026935e-02, -1.55368652e-02, -6.26485124e-02, -6.66373223e-03,
        1.43068153e-02, -4.92968187e-02,  1.30648721e-06, -2.02893210e-03,
       -4.47639599e-02,  2.07317546e-02, -2.80540297e-03,  1.37846796e-02,
        4.11506603e-03,  6.87661488e-03, -2.91272085e-02, -6.68382505e-03,
        2.94112526e-02, -4.13797721e-02,  3.90248671e-02, -5.73173016e-02,
        3.29415090e-02,  1.95522374e-03, -7.05714687e-04, -2.74959207e-02,
        8.47589783e-03,  

In [24]:
top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/CoderAgent/SecureAgent")

In [25]:
top_matches

{'matches': [{'id': '3429de0a-2314-483b-b937-cf5d64ad4d3e',
              'metadata': {'source': 'src/context/language/javascript-parser.ts',
                           'text': 'src/context/language/javascript-parser.ts\n'
                                   'import { AbstractParser, EnclosingContext '
                                   '} from "../../constants";\n'
                                   'import * as parser from "@babel/parser";\n'
                                   'import traverse, { NodePath, Node } from '
                                   '"@babel/traverse";\n'
                                   '\n'
                                   'const processNode = (\n'
                                   '  path: NodePath<Node>,\n'
                                   '  lineStart: number,\n'
                                   '  lineEnd: number,\n'
                                   '  largestSize: number,\n'
                                   '  largestEnclosingContext: Node | n

In [ ]:
system_prompt = f""""""


llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content